In [1]:
from PipelineTS.dataset import LoadWebSales

In [2]:
data = LoadWebSales()[['date', 'type_a']]
data.tail(5)

,date,type_a
2084,2021-09-09,8234
2085,2021-09-10,7825
2086,2021-09-11,8461
2087,2021-09-12,8159
2088,2021-09-13,8881


In [3]:
data = data.iloc[:-30, :]
valid_data = data.iloc[-30:, :]

# WideGBRT model

In [4]:
from PipelineTS.ml_model import WideGBRTModel

wide_gbrt = WideGBRTModel(time_col='date', target_col='type_a', lags=30, random_state=42)
wide_gbrt.fit(data)

In [7]:
wide_gbrt.predict(40)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-25,9845.032357,9822.804660,9867.260054
1,2021-08-26,9208.385637,9187.595334,9229.175940
2,2021-08-27,9086.939983,9066.423875,9107.456091
3,2021-08-28,10348.522843,10325.158387,10371.887299
4,2021-08-29,9476.127464,9454.732665,9497.522263
5,2021-08-30,9399.110885,9377.889971,9420.331800
6,2021-08-31,9630.933955,9609.189641,9652.678270
7,2021-09-01,9243.463037,9222.593538,9264.332537
8,2021-09-02,8918.230346,8898.095143,8938.365548
9,2021-09-03,9431.425672,9410.131799,9452.719545


# ProphetModel

In [4]:
from PipelineTS.statistic_model import ProphetModel
model = ProphetModel(time_col='date', target_col='type_a')

In [5]:
model.fit(data)

In [8]:
model.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,5173.562865,5119.488692,5227.637038
1,2021-08-16,9681.526466,9580.334976,9782.717956
2,2021-08-17,9195.977917,9099.861389,9292.094444
3,2021-08-18,8840.141283,8747.743966,8932.538600
4,2021-08-19,8579.255668,8489.585133,8668.926204
5,2021-08-20,8220.404602,8134.484784,8306.324420
6,2021-08-21,6249.091332,6183.775716,6314.406948
7,2021-08-22,6141.518392,6077.327130,6205.709653
8,2021-08-23,10773.515705,10660.910723,10886.120686
9,2021-08-24,10392.890130,10284.263454,10501.516806


# AutoARIMA

In [9]:
from PipelineTS.statistic_model import AutoARIMAModel

In [10]:
auto_arima = AutoARIMAModel(time_col='date', target_col='type_a')
auto_arima.fit(data=data)

In [11]:
auto_arima.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,8124.805268,8076.203340,8173.407196
1,2021-08-16,8518.231577,8467.276205,8569.186949
2,2021-08-17,8463.852404,8413.222324,8514.482484
3,2021-08-18,8334.845938,8284.987564,8384.704312
4,2021-08-19,8376.853656,8326.743995,8426.963317
5,2021-08-20,8416.138748,8365.794087,8466.483409
6,2021-08-21,8160.083454,8111.270495,8208.896413
7,2021-08-22,8294.867276,8245.248051,8344.486501
8,2021-08-23,8509.716657,8458.812220,8560.621094
9,2021-08-24,8509.771020,8458.866259,8560.675782


# GBDT

In [12]:
from PipelineTS.ml_model import *

## CatBoost

In [13]:
cbt = CatBoostModel(time_col='date', target_col='type_a', lags=30, random_state=42)
cbt.fit(data=data)

In [14]:
cbt.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,8957.702695,9186.877964,10061.656014
1,2021-08-16,9462.527744,9586.897964,10616.013265
2,2021-08-17,9488.463671,9985.364724,11674.546360
3,2021-08-18,9895.410365,10095.787030,11201.987898
4,2021-08-19,10079.628816,10103.018523,11519.962973
5,2021-08-20,10028.399849,9937.360584,11532.616508
6,2021-08-21,10513.704042,10072.580996,12000.855901
7,2021-08-22,10083.578903,10429.122653,11642.211980
8,2021-08-23,10300.888269,10760.687004,11178.251293
9,2021-08-24,10116.619723,9948.846896,11733.722030


## XGBoost

In [15]:
xgb = XGBoostModel(time_col='date', target_col='type_a', lags=30, random_state=42)
xgb.fit(data=data)

In [16]:
xgb.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,8109.658203,8084.823730,8134.492188
1,2021-08-16,9206.233398,9178.041016,9234.424805
2,2021-08-17,10129.718750,10098.698242,10160.738281
3,2021-08-18,9552.613281,9523.360352,9581.866211
4,2021-08-19,8632.674805,8606.239258,8659.110352
5,2021-08-20,9327.443359,9298.879883,9356.005859
6,2021-08-21,8350.327148,8324.755859,8375.897461
7,2021-08-22,10420.876953,10388.964844,10452.788086
8,2021-08-23,8265.765625,8240.453125,8291.077148
9,2021-08-24,10076.258789,10045.402344,10107.115234


## LightGBM

In [17]:
lgb = LightGBMModel(time_col='date', target_col='type_a', lags=30, random_state=42)
lgb.fit(data=data)

In [18]:
lgb.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,7076.917764,6749.944087,11246.670270
1,2021-08-16,8017.762986,7920.432959,11807.137600
2,2021-08-17,8956.557595,9590.323002,12826.493270
3,2021-08-18,8790.788577,9858.530543,13309.892457
4,2021-08-19,10205.794569,10185.427277,13193.163982
5,2021-08-20,10887.243250,9710.190276,13435.132552
6,2021-08-21,10670.619232,10065.626876,13644.494919
7,2021-08-22,10347.090470,9426.552115,13599.222823
8,2021-08-23,10073.151484,10347.311392,14030.416453
9,2021-08-24,10753.307191,9487.176561,13961.798461


## RandomForest

In [19]:
rf = RandomForestModel(time_col='date', target_col='type_a', lags=30, random_state=42)
rf.fit(data=data)

In [20]:
rf.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,9504.00,9481.120671,9526.879329
1,2021-08-16,9471.74,9448.938332,9494.541668
2,2021-08-17,9502.42,9479.544475,9525.295525
3,2021-08-18,9502.43,9479.554451,9525.305549
4,2021-08-19,9507.32,9484.432679,9530.207321
5,2021-08-20,9489.77,9466.924928,9512.615072
6,2021-08-21,9473.97,9451.162964,9496.777036
7,2021-08-22,9364.98,9342.435339,9387.524661
8,2021-08-23,9342.59,9320.099240,9365.080760
9,2021-08-24,9263.01,9240.710815,9285.309185


# NN Model

In [21]:
from PipelineTS.nn_model import *

## DLinearModel

In [22]:
d_linear = DLinearModel(time_col='date', target_col='type_a', lags=30, random_state=42)

In [23]:
d_linear.fit(data)
d_linear.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,8298.811523,8265.902344,8331.720703
1,2021-08-16,8857.418945,8822.294922,8892.542969
2,2021-08-17,8564.097656,8530.136719,8598.058594
3,2021-08-18,8490.714844,8457.044922,8524.384766
4,2021-08-19,8304.337891,8271.407227,8337.268555
5,2021-08-20,8040.516113,8008.631348,8072.400879
6,2021-08-21,7737.038574,7706.357422,7767.719727
7,2021-08-22,8021.276367,7989.468262,8053.084473
8,2021-08-23,8606.546875,8572.417969,8640.675781
9,2021-08-24,8470.290039,8436.701172,8503.878906


##  NLinearModel

In [24]:
n_linear = NLinearModel(time_col='date', target_col='type_a', lags=30, random_state=42)

In [25]:
n_linear.fit(data)
n_linear.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,8196.580078,8158.957031,8234.204102
1,2021-08-16,8604.582031,8565.085938,8644.078125
2,2021-08-17,8607.585938,8568.076172,8647.095703
3,2021-08-18,8335.010742,8296.751953,8373.269531
4,2021-08-19,8211.809570,8174.116699,8249.502930
5,2021-08-20,8035.030762,7998.149414,8071.912598
6,2021-08-21,7755.708008,7720.108398,7791.308105
7,2021-08-22,8179.750488,8142.204590,8217.296875
8,2021-08-23,8651.883789,8612.170898,8691.597656
9,2021-08-24,8497.446289,8458.442383,8536.451172


##  NBeatsModel 

In [26]:
n_beats = NBeatsModel(time_col='date', target_col='type_a', lags=30, random_state=42)

In [27]:
n_beats.fit(data)
n_beats.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,8327.192383,8264.924805,8389.459961
1,2021-08-16,8731.897461,8666.603516,8797.191406
2,2021-08-17,8579.774414,8515.618164,8643.930664
3,2021-08-18,8590.552734,8526.316406,8654.790039
4,2021-08-19,8878.488281,8812.098633,8944.878906
5,2021-08-20,8553.616211,8489.656250,8617.577148
6,2021-08-21,8055.523438,7995.287598,8115.759766
7,2021-08-22,8250.633789,8188.938965,8312.329102
8,2021-08-23,9136.042969,9067.727539,9204.359375
9,2021-08-24,8639.552734,8574.950195,8704.156250


## TCNModel

In [28]:
tcn = TCNModel(time_col='date', target_col='type_a', lags=30, random_state=42)

In [29]:
tcn.fit(data)
tcn.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,7910.128418,7877.528320,7942.728516
1,2021-08-16,7845.146484,7812.814453,7877.478516
2,2021-08-17,8464.758789,8429.873047,8499.644531
3,2021-08-18,8248.763672,8214.767578,8282.759766
4,2021-08-19,8290.397461,8256.230469,8324.564453
5,2021-08-20,8629.541016,8593.975586,8665.106445
6,2021-08-21,7846.062012,7813.726074,7878.397949
7,2021-08-22,7684.980957,7653.308594,7716.653320
8,2021-08-23,8132.227051,8098.711914,8165.742188
9,2021-08-24,8739.114258,8703.097656,8775.130859


## NHitsModel

In [30]:
n_hits = NHitsModel(time_col='date', target_col='type_a', lags=30, random_state=42)

In [31]:
n_hits.fit(data)
n_hits.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,7824.524902,7777.578125,7871.471680
1,2021-08-16,8459.645508,8408.888672,8510.402344
2,2021-08-17,8486.732422,8435.812500,8537.652344
3,2021-08-18,8430.742188,8380.158203,8481.326172
4,2021-08-19,8380.701172,8330.417969,8430.984375
5,2021-08-20,8595.738281,8544.164062,8647.312500
6,2021-08-21,8157.101562,8108.159668,8206.043945
7,2021-08-22,8217.091797,8167.790039,8266.393555
8,2021-08-23,8686.380859,8634.263672,8738.498047
9,2021-08-24,8842.955078,8789.898438,8896.011719


## TFTModel

In [32]:
tft = TFTModel(time_col='date', target_col='type_a', lags=30, random_state=42)

In [33]:
tft.fit(data)
tft.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-08-15,2209.738525,444.906128,3974.571045
1,2021-08-16,2209.738525,444.906128,3974.571045
2,2021-08-17,2209.738770,444.906189,3974.571533
3,2021-08-18,2209.738770,444.906189,3974.571533
4,2021-08-19,2209.738281,444.906067,3974.570557
5,2021-08-20,2209.738525,444.906128,3974.571045
6,2021-08-21,2209.738525,444.906128,3974.571045
7,2021-08-22,2209.738525,444.906128,3974.571045
8,2021-08-23,2209.738525,444.906128,3974.571045
9,2021-08-24,2209.738281,444.906067,3974.570557
